# Importing packages

In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import utils

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

import warnings
warnings.filterwarnings("ignore")

# Create dataframe

In [2]:
spotify_df = pd.read_csv('datasets/kaggleSPOTIFY.csv')

# Preprocessing

In [3]:
spotify_df.dropna(subset=['consolidates_genre_lists'], inplace=True)

In [4]:
spotify_df.isna().sum()

Unnamed: 0                  0
valence                     0
year                        0
acousticness                0
artists                     0
danceability                0
duration_ms                 0
energy                      0
explicit                    0
id                          0
instrumentalness            0
key                         0
liveness                    0
loudness                    0
mode                        0
name                        0
popularity                  0
release_date                0
speechiness                 0
tempo                       0
artists_upd_v1              0
artists_upd_v2              0
artists_upd                 0
artists_song                0
consolidates_genre_lists    0
dtype: int64

In [5]:
spotify_df.head()

,Unnamed: 0,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,...,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
0,0,0.177,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,...,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,['조정현'],[],['조정현'],조정현그아픔까지사랑한거야,['classic_korean_pop']
1,1,0.352,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,...,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,['黑豹'],[],['黑豹'],黑豹DON'T BREAK MY HEART,"['chinese_indie', 'chinese_indie_rock']"
2,2,0.458,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,...,藝旦調,23,1963-05-28,0.0443,85.936,['黃國隆'],[],['黃國隆'],黃國隆藝旦調,[]
3,3,0.796,1963,0.852,"['黃國隆', '王秋玉']",0.711,145720,0.1110,0,5xFXTvnEe03SyvFpo6pEaE,...,草螟弄雞公,23,1963-05-28,0.0697,124.273,"['黃國隆', '王秋玉']",[],"['黃國隆', '王秋玉']",黃國隆草螟弄雞公,[]
4,4,0.704,1963,0.771,['黃國隆'],0.610,208760,0.1750,0,6Pqs2suXEqCGx7Lxg5dlrB,...,思想起,23,1963-05-28,0.0419,124.662,['黃國隆'],[],['黃國隆'],黃國隆思想起,[]


In [6]:
mood_prep = spotify_df[['duration_ms', 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo']]

In [7]:
col_features = mood_prep.columns[:]
col_features

Index(['duration_ms', 'danceability', 'acousticness', 'energy',
       'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness',
       'tempo'],
      dtype='object')

In [8]:
mood_trans = MinMaxScaler().fit_transform(mood_prep[col_features])
mood_trans_np = np.array(mood_prep[col_features])

In [9]:
mood_trans_np[0]

array([ 2.37688e+05,  4.47000e-01,  5.68000e-01,  2.15000e-01,
        1.16000e-06,  6.49000e-02,  1.77000e-01, -1.64780e+01,
        2.72000e-02,  7.19790e+01])

In [10]:
df = pd.read_csv('datasets/data_moods.csv')

In [11]:
cl_features = df.columns[6:-3]
X= MinMaxScaler().fit_transform(df[cl_features])
X2 = np.array(df[cl_features])
Y = df['mood']

In [12]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


dummy_y = utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

,mood,encode
5,Calm,0
4,Energetic,1
0,Happy,2
1,Sad,3


# Model creation

In [13]:
def base_model():
    model = Sequential()
    model.add(Dense(8,input_dim=10,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [14]:
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)

In [18]:
kfold = KFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X,encoded_y,cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

Baseline: 79.59% (3.09%)


In [16]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict(X_train)

In [17]:
pip = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
pip.fit(X2,encoded_y)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('keras',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000000AC64B45B20>)])

# Predict & Add Mood Column

In [34]:
def predict_mood(preds):
    
    preds_features = np.array(preds[:]).reshape(-1,1).T

    results = pip.predict(preds_features)

    mood = np.array(target['mood'][target['encode']==int(results)])

    return str(mood[0])

In [ ]:
res = []

for i in range(len(mood_trans_np)):
  res.append(predict_mood(mood_trans_np[i]))

In [ ]:
spotify_df['Mood'] = np.resize(res,len(spotify_df))

In [ ]:
spotify_df.to_csv('kaggleMusicMoodFinal.csv')